# Imports

In [1]:
from qiskit.circuit.library.standard_gates import RXGate, RZGate, RYGate, CXGate, CZGate, SGate, HGate
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from typing import Dict, Tuple, List
import numpy as np
from qiskit.opflow import X, Z, I, H
from qiskit.quantum_info import Pauli
import matplotlib
from qiskit import assemble, Aer
from qiskit import *
from qiskit.visualization import plot_histogram
import math
from qiskit.algorithms.optimizers import L_BFGS_B

# Implementing the ansatz from qiskit tutorial

In [2]:
def calc_linear_entangelment_ansatz(num_of_qubits, thetas, depth):
    if thetas.size != 2*num_of_qubits*(depth + 1):
        print("ERROR with thetas size")
    else:
        qr = QuantumRegister(num_of_qubits, name="qubit")
        qc = QuantumCircuit(qr)
        
        for iteration in range(depth):
            for qubit_index in range(num_of_qubits):
                RY_theta_index = iteration*2*num_of_qubits + qubit_index
                RZ_theta_index = RY_theta_index + num_of_qubits
                qc.append(RYGate(thetas[RY_theta_index]), [qr[qubit_index]])
                qc.append(RZGate(thetas[RZ_theta_index]), [qr[qubit_index]])
            for qubit_index in range(num_of_qubits - 1):
                qc.append(CXGate(), [qr[qubit_index], qr[qubit_index + 1]])
        
        for qubit_index in range(num_of_qubits):
                RY_theta_index = 2*num_of_qubits*depth + qubit_index
                RZ_theta_index = RY_theta_index + num_of_qubits
                qc.append(RYGate(thetas[RY_theta_index]), [qr[qubit_index]])
                qc.append(RZGate(thetas[RZ_theta_index]), [qr[qubit_index]])
        
        return qc

def calc_full_entangelment_ansatz(num_of_qubits, thetas, depth):
    if thetas.size != 2*num_of_qubits*(depth + 1):
        print("ERROR with thetas size")
    else:
        qr = QuantumRegister(num_of_qubits, name="qubit")
        qc = QuantumCircuit(qr)
        
        for iteration in range(depth):
            for qubit_index in range(num_of_qubits):
                RY_theta_index = iteration*2*num_of_qubits + qubit_index
                RZ_theta_index = RY_theta_index + num_of_qubits
                qc.append(RYGate(thetas[RY_theta_index]), [qr[qubit_index]])
                qc.append(RZGate(thetas[RZ_theta_index]), [qr[qubit_index]])
            for qubit_index in range(num_of_qubits - 1):
                for target_qubit_index in range(qubit_index + 1, num_of_qubits):
                    qc.append(CXGate(), [qr[qubit_index], qr[target_qubit_index]])
        
        for qubit_index in range(num_of_qubits):
                RY_theta_index = 2*num_of_qubits*depth + qubit_index
                RZ_theta_index = RY_theta_index + num_of_qubits
                qc.append(RYGate(thetas[RY_theta_index]), [qr[qubit_index]])
                qc.append(RZGate(thetas[RZ_theta_index]), [qr[qubit_index]])
        
        return qc

def calc_ansatz_circ(num_of_qubits, entangelment, thetas, depth):
    if entangelment == "linear":
        return calc_linear_entangelment_ansatz(num_of_qubits, thetas, depth)
    assert entangelment == "full", "entangelment has to be linear or full"
    return calc_full_entangelment_ansatz(num_of_qubits, thetas, depth)

# Generating the hamiltonians

In [3]:
H2_molecule_Hamiltonian = -0.5053051899926562*(I^I) + \
                            -0.3277380754984016*(Z^I) + \
                            0.15567463610622564*(Z^Z) + \
                            -0.3277380754984016*(I^Z)

def create_pauli_string_with_pauli_op_on_index_i(pauli_op, i, qubits_num):
    if i == 1:
        pauli_string = pauli_op
        for qubit in range(qubits_num - 1):
            pauli_string = pauli_string ^ I
        return pauli_string
    
    pauli_string = I
    for qubit in range(2, qubits_num + 1):
        if qubit == i:
            pauli_string = pauli_string ^ pauli_op
        else:
            pauli_string = pauli_string ^ I
            
    return pauli_string

def create_pauli_string_with_pauli_ops_on_index_i_and_j(pauli_op_second, i, pauli_op_first, j, qubits_num):
    if j == 1:
        pauli_string = pauli_op_first
        for qubit in range(2, qubits_num + 1):
            if qubit == i:
                pauli_string = pauli_string ^ pauli_op_second
            else:
                pauli_string = pauli_string ^ I
        return pauli_string
    
    pauli_string = I
    for qubit in range(2, qubits_num + 1):
        if qubit == j:
            pauli_string = pauli_string ^ pauli_op_first
        elif qubit == i:
            pauli_string = pauli_string ^ pauli_op_second
        else:
            pauli_string = pauli_string ^ I
            
    return pauli_string
    
def get_Ising_model_hamiltonian():
    hamiltonian = I
    for qubit in range(QUBITS_NUM - 1):
        hamiltonian = hamiltonian^I
    hamiltonian = 0 * hamiltonian
    
    for i in range(1, QUBITS_NUM + 1):
        a_i = np.random.random_sample()
        x_i = create_pauli_string_with_pauli_op_on_index_i(X, i, QUBITS_NUM)
        hamiltonian = hamiltonian + a_i * x_i
        for j in range(1, i):
            J_ij = np.random.random_sample()
            z_ij = create_pauli_string_with_pauli_ops_on_index_i_and_j(Z, i, Z, j, QUBITS_NUM)
            hamiltonian = hamiltonian + J_ij * z_ij
    
    return hamiltonian
    
def get_hamiltonian(hamiltonian_type):
    if hamiltonian_type == "Ising Model":
        return get_Ising_model_hamiltonian()
    assert hamiltonian_type == "Molecular", "hamiltonian_type has to be Ising Model or Molecular"
    return H2_molecule_Hamiltonian

# Expectation Value

In [4]:
reducing_to_pauli_z_dict = {
    Pauli('I'): Pauli('I'),
    Pauli('Z'): Pauli('Z'),
    Pauli('X'): Pauli('Z'),
    Pauli('Y'): Pauli('Z')
} 

def transfrom_hamiltonian_into_pauli_string(hamiltonian):
    pauli_operators = hamiltonian.to_pauli_op().settings['oplist']
    pauli_strings = list(map(lambda pauli_operator: pauli_operator.primitive, pauli_operators))
    pauli_coeff = list(map(lambda pauli_operator: pauli_operator.coeff, pauli_operators))
    return (pauli_strings, pauli_coeff)

def reduce_pauli_matrixes_into_sigma_z(pauli_string):
    for matrix_index in range(QUBITS_NUM):
        pauli_matrix = pauli_string[matrix_index]
        pauli_string[matrix_index].insert(reducing_to_pauli_z_dict[pauli_matrix])
    
    print(pauli_string)
    return pauli_string

def get_z_reduction_for_pauli_string(qc, pauli_string):
    qr = QuantumRegister(QUBITS_NUM, name="q")
    exdend_qc = QuantumCircuit(qr)
    pauli_string = str(pauli_string)
    for qubit_index, pauli_matrix in enumerate(pauli_string):
        if pauli_matrix == "X":
            exdend_qc.append(HGate(), [qr[qubit_index]])
        elif pauli_matrix == "Y":
            exdend_qc.append(HGate(), [qr[qubit_index]])
            exdend_qc.append(SGate(), [qr[qubit_index]])
    qc = qc.compose(exdend_qc)
    return qc

def get_probability_distribution(counts: Dict) -> Dict:
    proba_distribution = {state: (count / NUM_SHOTS) for state, count in counts.items()}
    return proba_distribution

def calculate_probabilities_of_measurments_in_computational_basis(quantum_state_circuit) -> Dict:
    quantum_state_circuit.measure_all()

    transpiled_quantum_state_circuit = transpile(quantum_state_circuit, simulator_backend) 
    Qobj = assemble(transpiled_quantum_state_circuit)
    result = simulator_backend.run(Qobj).result() 
    counts = result.get_counts(quantum_state_circuit)
    
    return get_probability_distribution(counts)

def sort_probas_dict_by_qubits_string_keys(proba_distribution: Dict) -> Dict:
    return dict(sorted(proba_distribution.items()))

def reset_power_of_minus_1(power_of_minus_1):
    power_of_minus_1 = 0
    return power_of_minus_1

def calculate_expectation_value_of_pauli_string_by_measurments_probas(pauli_string, probas_distribution):
    pauli_string_expectation_value = 0
    power_of_minus_1 = 0
    
    sorted_probas_distribuition = sort_probas_dict_by_qubits_string_keys(probas_distribution)
    for qubits_string, proba in sorted_probas_distribuition.items():
        for string_index in range(QUBITS_NUM):
            if(str(qubits_string[string_index])=="1" and str(pauli_string[string_index])=="Z"):
                power_of_minus_1 += 1
            
        pauli_string_expectation_value += pow(-1, power_of_minus_1)*proba
        power_of_minus_1 = reset_power_of_minus_1(power_of_minus_1)
        
    return pauli_string_expectation_value

def get_expectation_value(pauli_string, probas_distribution):
        return calculate_expectation_value_of_pauli_string_by_measurments_probas(
                                                                                pauli_string, probas_distribution)

# Calculating the target function

In [5]:
def target_function(thetas):
    ansatz = calc_ansatz_circ(QUBITS_NUM, Entangelment, thetas, Depth)
    
    qr = QuantumRegister(QUBITS_NUM, name="qubit")
    qc = QuantumCircuit(qr)
    qc.initialize(Init_state, qc.qubits)
    
    total_expectation_value = 0
    pauli_strings, pauli_coeffs = transfrom_hamiltonian_into_pauli_string(Ham)
    for pauli_index, pauli_string in enumerate(pauli_strings):
        qc_per_pauli_string = qc.compose(ansatz)
        qc_per_pauli_string = get_z_reduction_for_pauli_string(qc_per_pauli_string, pauli_string)
        probas_distribution = calculate_probabilities_of_measurments_in_computational_basis(qc_per_pauli_string)
        total_expectation_value += pauli_coeffs[pauli_index] * get_expectation_value(pauli_string, probas_distribution)
    print("total expectation value:")
    print(total_expectation_value)
    return total_expectation_value
        

# Implementing the VQE algorithem

In [6]:
def VQE(optimazer, initial_thetas):
    point, value, nfev = optimazer.optimize(QUBITS_NUM*2*(Depth + 1), target_function, initial_point=initial_thetas)
    base_energy = value
    print("The VQE result for the minimal energy of the Hamiltonian given is:")
    print(base_energy)

# Setting the simulator and optimazer

In [7]:
bfgs_optimizer = L_BFGS_B(maxiter=2)

simulator_backend = Aer.get_backend('aer_simulator') 

# VQE with Transverse Ising Model Hamiltonian + LINEAR Entangelment Ansatz

In [9]:
#Backend number of shots
NUM_SHOTS = 1024

QUBITS_NUM = 4
Ham = get_hamiltonian("Ising Model")
Entangelment = "linear"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
Stop_optimization_threshold = 100
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = bfgs_optimizer
VQE(optimazer, initial_thetas)
    

total expectation value:
1.5268521521809366
total expectation value:
1.484271342383039
total expectation value:
1.5247045038138227
total expectation value:
1.5583152206737556
total expectation value:
1.5242985777349587
total expectation value:
1.5747117913700344
total expectation value:
1.5460220935457927
total expectation value:
1.515257768599987
total expectation value:
1.60165657704944
total expectation value:
1.514681308216236
total expectation value:
1.50311281244624
total expectation value:
1.5485166810862057
total expectation value:
1.5456335852317071
total expectation value:
1.5688880867918518
total expectation value:
1.524480155375731
total expectation value:
1.5038246200068082
total expectation value:
1.5209426189138686
total expectation value:
1.528575042884956
total expectation value:
1.4828180231149535
total expectation value:
1.5398177897505136
total expectation value:
1.564872731674399
total expectation value:
1.5997585990155925
total expectation value:
1.514630387033649

total expectation value:
1.4772821915184222
total expectation value:
1.4811009024962023
total expectation value:
1.5785630906073391
total expectation value:
1.6547316772461396
total expectation value:
1.5196586217604913
total expectation value:
1.5633721620735448
total expectation value:
1.5290649532330978
total expectation value:
1.4746628553418104
total expectation value:
1.567178191568389
total expectation value:
1.4720484367728108
total expectation value:
1.4919327798643245
total expectation value:
1.4547668283064012
total expectation value:
1.5544839160092974
total expectation value:
1.5733587544023062
total expectation value:
1.5073862271225136
total expectation value:
1.5224619196317457
total expectation value:
1.60288740119091
total expectation value:
1.5784222897063662
total expectation value:
1.5499525819379916
total expectation value:
1.471559339491009
total expectation value:
1.6046089632966565
total expectation value:
1.5491623596569097
total expectation value:
1.524161233

total expectation value:
1.526517812256976
total expectation value:
1.5187898177544397
total expectation value:
1.5862523122421315
total expectation value:
1.592490281580767
total expectation value:
1.5586379820897063
total expectation value:
1.5543442804100418
total expectation value:
1.5449199514685743
total expectation value:
1.5448441349915196
total expectation value:
1.562656351833739
total expectation value:
1.5634846317271283
total expectation value:
1.5455756223612005
total expectation value:
1.5814337117617154
total expectation value:
1.4908883279124971
total expectation value:
1.5424422614721505
total expectation value:
1.4897532014942856
total expectation value:
1.5622709676780853
total expectation value:
1.59578465393476
total expectation value:
1.592935443455072
total expectation value:
1.5843447069346286
total expectation value:
1.460111231349783
total expectation value:
1.5680123779871646
total expectation value:
1.5870416340358195
total expectation value:
1.562908328610

total expectation value:
1.5354567569672888
total expectation value:
1.526804988350486
total expectation value:
1.5878864265294337
total expectation value:
1.4781276889445814
total expectation value:
1.5574841896990397
total expectation value:
1.5896527826787565
total expectation value:
1.6535515242335705
total expectation value:
1.4798912141384581
total expectation value:
1.4905306604260933
total expectation value:
1.5413790847116746
total expectation value:
1.5407216460050608
total expectation value:
1.5600925547875264
total expectation value:
1.6162274142430677
total expectation value:
1.5058238469308716
total expectation value:
1.5247247115451645
total expectation value:
1.5117414880142572
total expectation value:
1.531342604064099
total expectation value:
1.5330592956048414
total expectation value:
1.5307145508171294
total expectation value:
1.559181966561549
total expectation value:
1.551591448887231
total expectation value:
1.5388585109493935
total expectation value:
1.569903486

total expectation value:
1.5634407094125589
total expectation value:
1.5876958937072243
total expectation value:
1.552845781729927
total expectation value:
1.5417264608198233
total expectation value:
1.4551150036796372
total expectation value:
1.5638357467188855
total expectation value:
1.5397773942425759
total expectation value:
1.5733588881963823
The VQE result for the minimal energy of the Hamiltonian given is:
1.4696646579898076


# VQE with Transverse Ising Model Hamiltonian + FULL Entangelment Ansatz

In [ ]:
#Backend number of shots
NUM_SHOTS = 1024

QUBITS_NUM = 4
Ham = get_hamiltonian("Ising Model")
Entangelment = "full"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
Stop_optimization_threshold = 100
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = bfgs_optimizer
VQE(optimazer, initial_thetas)

total expectation value:
0.9808950441058699
total expectation value:
0.945865628449865
total expectation value:
0.940341507588736
total expectation value:
0.8944461989661848
total expectation value:
0.9421929478362681
total expectation value:
0.9030911642591601
total expectation value:
0.8857336468095868
total expectation value:
0.9786141187721026
total expectation value:
0.8166366058723423
total expectation value:
0.9879703954717833
total expectation value:
0.9682517494297793
total expectation value:
0.9808447895788305
total expectation value:
0.9214229539340343
total expectation value:
0.8733965968521948
total expectation value:
0.9532942384779319
total expectation value:
0.9222033498541307
total expectation value:
0.9039705220679188
total expectation value:
0.923650365307329
total expectation value:
0.9681951811013749
total expectation value:
0.9206802198474584
total expectation value:
0.9757591044641034
total expectation value:
0.9952036619811752
total expectation value:
0.94494653

# VQE with H2 Molecule Hamiltonian + LINEAR Entangelment Ansatz

In [ ]:
#Backend number of shots
NUM_SHOTS = 1024

QUBITS_NUM = 2
Ham = get_hamiltonian("Molecular")
Entangelment = "linear"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
Stop_optimization_threshold = 100
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = bfgs_optimizer
VQE(optimazer, initial_thetas)

# VQE with H2 Molecule Hamiltonian + FULL Entangelment Ansatz

In [ ]:
#Backend number of shots
NUM_SHOTS = 1024

QUBITS_NUM = 2
Ham = get_hamiltonian("Molecular")
Entangelment = "full"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
Stop_optimization_threshold = 100
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = bfgs_optimizer
VQE(optimazer, initial_thetas)